In [2]:
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv("retouren.csv")

# 1. Data Cleaning

In [4]:
df.describe().round(1)

,order_item_id,item_id,brand_id,item_price,user_id,return
count,75007.0,75007.0,75007.0,75007.0,75007.0,75007.0
mean,37504.0,784.8,28.8,60.7,25740.2,0.5
std,21652.8,680.0,26.8,44.7,12947.2,0.5
min,1.0,1.0,1.0,0.0,9.0,0.0
25%,18752.5,176.0,5.0,29.9,14674.0,0.0
50%,37504.0,479.0,24.0,49.9,30486.0,0.0
75%,56255.5,1517.0,43.0,74.9,36479.0,1.0
max,75007.0,2224.0,137.0,999.0,43801.0,1.0


In [5]:
df.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.90,30822,Mrs,1969-04-17,Saxony,2016-06-23,0
1,2,2016-06-22,NaN,337,152,grey,30,19.95,30822,Mrs,1969-04-17,Saxony,2016-06-23,0
2,3,2016-06-22,2016-06-27,270,xxl,grey,49,79.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,1
3,4,2016-06-22,2016-06-27,142,xxl,grey,49,99.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,0
4,5,2016-06-22,2016-06-27,561,xxl,grey,3,14.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,1


### Dtypes

In [6]:
df.dtypes

order_item_id      int64
order_date        object
delivery_date     object
item_id            int64
item_size         object
item_color        object
brand_id           int64
item_price       float64
user_id            int64
user_title        object
user_dob          object
user_state        object
user_reg_date     object
return             int64
dtype: object

- **order_date, delivery_date, user_dob als datetime**

In [7]:
df.delivery_date = pd.to_datetime(df.delivery_date)
df.order_date = pd.to_datetime(df.order_date)
df.user_dob = pd.to_datetime(df.user_dob)
df.user_reg_date = pd.to_datetime(df.user_reg_date)

### Item_id's mit item_price von 0 

In [8]:
df[df.item_price == 0].item_id.unique()

array([ 157,   62,  312,  553,  807, 1014,  239, 1436, 1725, 1904, 1951,
       1472, 2025, 2103, 2121, 2199, 2213])

In [9]:
df[df.item_id == 157][["item_color", "item_price"]]

,item_color,item_price
202,purple,0.0
216,purple,0.0
218,purple,0.0
329,purple,9.9
351,purple,0.0
...,...,...
65632,jade,9.9
65943,jade,9.9
66731,jade,9.9
68026,jade,9.9


In [10]:
df[df.item_id == 157][["item_color", "item_price"]].groupby("item_color").agg(
 mean_price = ("item_price", "mean"),
 max_price= ("item_price", "max"),
 min_price = ("item_price", "min")
)

,mean_price,max_price,min_price
item_color,,,
jade,10.710811,19.9,9.9
purple,1.218436,19.9,0.0


In [11]:
df[(df.item_id == 157) & (df.item_color == "purple")][["item_color", "item_price"]].item_price.unique()

array([ 0. ,  9.9, 19.9])

In [12]:
df[df.item_id == 157][["item_color", "item_price"]].groupby(["item_color", "item_price"]).agg(
 item_price_count = ("item_price", "count")
)

item_price_count
item_color item_price                  
jade       9.9                       34
           19.9                       3
purple     0.0                      160
           9.9                       16
           19.9                       3

- **für die item_id 157 gibt es verschiedene Preise** 
- **Promotion Angebot zB. 2 für 3**

-> **nicht in df ändern**


### Delivery_date NaN

In [13]:
df[df.delivery_date.isna()][["order_date", "delivery_date", "user_id"]]

,order_date,delivery_date,user_id
1,2016-06-22,NaT,30822
55,2016-06-23,NaT,30828
63,2016-06-23,NaT,22948
64,2016-06-23,NaT,22948
65,2016-06-23,NaT,22948
...,...,...,...
74831,2016-08-20,NaT,43763
74839,2016-08-20,NaT,38972
74924,2016-08-20,NaT,19160
74925,2016-08-20,NaT,19160


In [14]:
df[df.delivery_date.isna()][["order_date", "delivery_date", "user_id"]].nunique()

order_date         60
delivery_date       0
user_id          4048
dtype: int64

In [15]:
df_1 = df.copy()

In [16]:
df_1[df_1.delivery_date.isna()][["order_date", "delivery_date", "user_id"]].user_id.unique()

array([30822, 30828, 22948, ..., 16813, 43763, 19160])

In [17]:
df_1.delivery_date = df_1.delivery_date.fillna("not delivered")
df_1.delivery_date

0        2016-06-27 00:00:00
1              not delivered
2        2016-06-27 00:00:00
3        2016-06-27 00:00:00
4        2016-06-27 00:00:00
                ...         
75002    2016-08-21 00:00:00
75003    2016-08-21 00:00:00
75004    2016-08-23 00:00:00
75005    2016-09-07 00:00:00
75006    2016-09-07 00:00:00
Name: delivery_date, Length: 75007, dtype: object

In [18]:
df_d = df_1.groupby(["user_id", "order_date", "delivery_date", "return"]).agg(
 count = ("delivery_date", "count")
).reset_index()
df_d

,user_id,order_date,delivery_date,return,count
0,9,2016-08-01,2016-08-02 00:00:00,1,2
1,9,2016-08-01,2016-08-24 00:00:00,1,1
2,9,2016-08-10,2016-08-14 00:00:00,0,1
3,9,2016-08-10,2016-08-14 00:00:00,1,1
4,11,2016-07-02,2016-07-03 00:00:00,0,1
...,...,...,...,...,...
38988,43799,2016-08-20,2016-08-21 00:00:00,0,2
38989,43801,2016-08-20,2016-08-21 00:00:00,0,5
38990,43801,2016-08-20,2016-08-21 00:00:00,1,5
38991,43801,2016-08-20,2016-08-23 00:00:00,1,1


In [19]:
df_nan = df_d[df_d.delivery_date == "not delivered"].user_id
df_nan

13          55
23          60
27          66
43          94
60         104
         ...  
38889    43707
38902    43717
38915    43725
38920    43728
38953    43763
Name: user_id, Length: 4691, dtype: int64

In [20]:
df_d[df_d.user_id.isin(df_nan)]

,user_id,order_date,delivery_date,return,count
11,55,2016-06-29,2016-07-03 00:00:00,0,1
12,55,2016-06-29,2016-07-03 00:00:00,1,1
13,55,2016-06-29,not delivered,0,1
14,55,2016-08-07,2016-08-08 00:00:00,0,2
15,55,2016-08-07,2016-08-08 00:00:00,1,6
...,...,...,...,...,...
38918,43728,2016-08-19,2016-08-24 00:00:00,1,4
38919,43728,2016-08-19,2016-08-30 00:00:00,1,1
38920,43728,2016-08-19,not delivered,0,3
38952,43763,2016-08-20,2016-08-23 00:00:00,0,1


- **für NaN werte bei delivery_date not delivered eintragen (behalten)** 
- **falls das Modell vor der Bestellung eingesetzt werden soll spielt das delivery_date noch keine Rolle (weglassen)**  

In [21]:
df_dd = df_1.groupby(["user_id", "user_state", "delivery_date"]).agg(
 count = ("user_id", "count")
).reset_index()

In [22]:
df_ddd = df_dd[df_dd.user_id.isin(df_nan)]

In [23]:
df_ddd.user_state.unique()

array(['Rhineland-Palatinate', 'North Rhine-Westphalia', 'Bavaria',
       'Lower Saxony', 'Schleswig-Holstein', 'Saxony',
       'Baden-Wuerttemberg', 'Saarland', 'Saxony-Anhalt', 'Hesse',
       'Berlin', 'Thuringia', 'Hamburg', 'Brandenburg', 'Bremen',
       'Mecklenburg-Western Pomerania'], dtype=object)

In [24]:
df_pie= df_ddd.groupby(["user_id", "user_state"]).agg(count = ("user_id", "count")).reset_index()

In [25]:
px.pie(df_pie, names= "user_state", values= "count")

### Quantile bei age 

In [26]:
df.age = df.order_date.dt.year - df.user_dob.dt.year - ((df.order_date.dt.date)<(df.user_dob.dt.date))

/var/folders/f6/gs3z1yx9005b5dmpnd_z3t3m0000gn/T/ipykernel_64199/267430987.py:1: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



In [27]:
df.age.quantile(0.984)

79.0

In [28]:
df.age.quantile(0.00055)

17.0

In [29]:
df = df.loc[df.age.between(df.age.quantile(0.00055), df.age.quantile(0.984))]

### Item_size anpassen

- **Werte wie 152 könnten Kinderkleidung sein**
- **Werte wie 4032 sind Hosengrößen, könnte man aufspalten**
- **Werte mit + könnten markieren, dass das Produkt stretchy ist** 

In [30]:
df.item_size.unique()

array(['38', '152', 'xxl', '41', '50', '48', 'xl', '42', '43', '9', '8+',
       'unsized', 'm', 's', 'l', '40', '40+', '6+', '10', '8', '7', '44',
       '37', '39', '36', '4+', '21', '7+', '20', '18', '5+', '46', '6',
       '19', '4', '5', '35', '31', '38+', '29', '34', '3', '30', '45',
       '22', '24', '41+', '39+', '23', '140', '116', '26', '3632', '4032',
       '10+', '1', '9+', '36+', '11', '28', '11+', '12', '25', '37+',
       '128', '176', 'xs', '42+', '3432', '164', '52', '104', '33', '32',
       '14', '27', '13', '56', '3+', '54', 'xxxl', '3332', '46+', '2',
       '47', '80', '90', '3832', '3634', '44+', '49', '43+', '2+', '45+',
       '100', '3132', '58', '4034'], dtype=object)

- **Größen One Hot Encoden**
- **für Hosen Hüftbreite und Läng aufspalten**

### Accountalter der User hinzufügen als extra Spalte im df

In [59]:
df["acc_age"] = df.order_date - df.user_reg_date 

In [ ]:
df["acc_age"] = df.acc_age.dt.days

In [62]:
df

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,acc_age,acc_age1
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.90,30822,Mrs,1969-04-17,Saxony,2016-06-23,0,-1 days,-1
1,2,2016-06-22,NaT,337,152,grey,30,19.95,30822,Mrs,1969-04-17,Saxony,2016-06-23,0,-1 days,-1
2,3,2016-06-22,2016-06-27,270,xxl,grey,49,79.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,1,465 days,465
3,4,2016-06-22,2016-06-27,142,xxl,grey,49,99.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,0,465 days,465
4,5,2016-06-22,2016-06-27,561,xxl,grey,3,14.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,1,465 days,465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75002,75003,2016-08-20,2016-08-21,1447,39,brown,27,6.95,43801,Mrs,1961-11-25,Mecklenburg-Western Pomerania,2016-08-21,0,-1 days,-1
75003,75004,2016-08-20,2016-08-21,2155,39,white,27,14.95,43801,Mrs,1961-11-25,Mecklenburg-Western Pomerania,2016-08-21,1,-1 days,-1
75004,75005,2016-08-20,2016-08-23,1473,44,black,7,19.90,43801,Mrs,1961-11-25,Mecklenburg-Western Pomerania,2016-08-21,1,-1 days,-1
75005,75006,2016-08-20,2016-09-07,1784,unsized,olive,41,19.90,43801,Mrs,1961-11-25,Mecklenburg-Western Pomerania,2016-08-21,0,-1 days,-1
